# Laboratorio de regresión logística

|                |   |
:----------------|---|
| **Juan Jorge Camarena**     |   |
| **2/10/24**      |   |
| **746176** |   | 

La regresión logística es una herramienta utilizada para predecir respuestas cualitativas. Al igual que la regresión lineal, es un método sencillo que sirve como un punto de partida para técnicas más avanzadas. Por ejemplo, lo que se conoce como *redes neuronales* o *red de perceptrones multicapa* no es más que una estructura de regresiones logísticas que se alimentan entre sí.

1. Descarga el archivo de créditos y carga los datos (Default.csv). Utiliza `pandas`.

In [ ]:
import pandas as pd

In [ ]:
datos = pd.read_csv("Default.csv")

In [ ]:
datos

2. Utiliza el comando `obj.head()`, donde `obj` es el nombre que le diste a los datos del archivo.

In [ ]:
datos.head()

El comando head arroja los primeras *n* líneas (por defecto 5) de los datos que están en el DataFrame.

3. Utiliza el comando `obj.describe()`.

In [ ]:
datos.describe()

El comando describe toma las columnas que tienen datos numéricos y saca datos estadísticos comunes:
- *n*
- media
- desviación estándar
- valor mínimo
- primer cuartil
- mediana
- tercer cuartil
- valor máximo

3. Vistos estos datos, ¿qué columnas existen en el DataFrame? ¿Qué tipo de datos contienen?

El balance en una cuenta de banco y los ingresos de los dueños de las cuentas

4. Configura el tipo de dato de las columnas `default` y `student` para cambiarlos a variables categóricas.

`data[columna] = data[columna].astype("category")`

In [ ]:
datos["default"] = datos["default"].astype("category")

In [ ]:
datos["student"] = datos["student"].astype("category")

In [ ]:
datos["default"], datos["student"]

Imagina que trabajas en un banco y que se te entregan estos datos. Tu objetivo es crear un modelo que ayude a predecir si una persona que solicita un crédito lo va a pagar. Exploremos los datos un poco más antes de crear un modelo.

Veamos primero cómo es la distribución de los valores cuando una persona dejó de pagar y cuando siguió pagando. `Default` es el término utilizado para cuando una persona dejó de pagar.

5. Crea una gráfica de caja para las columnas `income` y `balance`, con los datos agrupados con la columna `default`. Utiliza el comando `obj.boxplot(column=____, by=_____)`

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
datos.boxplot(column='income', by='default')
plt.xlabel('Default')
plt.ylabel('Income')
plt.show()

In [ ]:
datos.boxplot(column='balance', by='default')
plt.xlabel('Default')
plt.ylabel('Balance')
plt.show()

6. Crea una gráfica de dispersión donde el eje *x* sea la columna `balance` y el eje *y* la columna `income`. Utiliza el comando `obj.plot.scatter(x, y, c="default", colormap="PiYG_r", alpha=0.5)`.

In [ ]:
datos.plot.scatter(x="balance", y="income", c="default", colormap="PiYG_r", alpha=.05)
plt.xlabel("Balance")
plt.ylabel("Income")
plt.show()

La regresión (lineal o logística) se usa para encontrar una línea que ajuste los datos para tomar una decisión. La línea que buscamos en regresión logística es aquella que nos ayude a separar las diferentes categorías. 

<img style="float: left; " src="https://www.baeldung.com/wp-content/uploads/sites/4/2023/10/decision_boundary_curve.jpg" width="400px" />


## Regresión logística simple

Creemos un modelo simple donde sólo utilizamos una de los factores para predecir una respuesta. Quiero conocer la probabilidad de que una persona deje de pagar su crédito dado el balance que tiene en su cuenta.

$$ P(\text{default}=\text{Yes}|\text{balance}) $$

Por el momento la columna default no contiene valores numéricos, por lo que hay que transformar los datos. Como default es nuestra variable de respuesta (lo que queremos predecir) podemos nombrarla *y*.

Ejecuta el código `y = obj["default"] == "Yes"`. Extrae el factor `balance` en una variable *x*.

In [ ]:
y = datos["default"] == "Yes"
x = datos["balance"]

Crea un gráfico de dispersión donde el eje *x* sea `balance` y el eje *y* sea `default` transformado.

In [ ]:
#chat

datos["default"].map({"No": 0, "Yes": 1})
x = datos["balance"]
y = datos["default"]
plt.scatter(x, y, alpha=0.5)
plt.xlabel("Balance")
plt.ylabel("Default")
plt.show()

La línea que utilizaremos para predecir la probabilidad es:

$$ p(x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}} $$

Para nuestro ejemplo de pagos y balance:

$$ P(\text{default}=1|\text{balance}) = \frac{1}{1 + e^{-(\beta_0 + \beta_1  \text{balance})}} $$

Buscamos maximizar la probabilidad de que el modelo tome decisiones correctas. Es decir, que cuando `default` fue verdadero, que la predicción sea 100%, y que cuando `default` fue falso que la predicción sea 0%.

$$ \Pi_{i:y_i=1} p(x_i) \Pi_{i':y_{i'}} (1-p(x_{i'})) $$

La función de costo ya simplificada es la siguiente:

$$ J(\vec{\beta}) = -  \sum_{i=1}^n{[y_i \ln{(\hat{p}(x_i))} + (1-y_i)\ln{(1 - \hat{p}(x_i))}]}$$

Utiliza la función `minimize` del módulo `optimize` la librería `scipy` para estimar los parámetros del modelo. Utiliza $\beta_0=-10$ y $\beta_1=0.005$ como valores iniciales.

In [ ]:
import scipy.optimize as opt
import numpy as np

In [ ]:
n = len(y)
ones = np.ones([n, 1])
balance = np.reshape((x),[n, 1])
X = np.hstack((ones, balance))
y_vector = np.reshape(y, [n, 1])

def pr(beta, X) :
    return 1/(1 + np.exp(-X @ beta))

def J(beta, X, y):
    epsilon = 1e-12
    n, p = X.shape
    beta = np.reshape(beta,[p, 1])
    y_pred = pr(beta, X)
    e = y*np.log(y_pred+epsilon)+(1-y)*np.log(1-y_pred+epsilon)
    return -np.sum(e)
beta = [-10, 0.005]
rss = opt.minimize(J, beta, args=(X, y_vector))
rss

Muchos aspectos de la regresión logística son similares a la regresión lineal. Podemos medir la precisión de nuestros estimados calculando sus errores estándar. El objetivo de calcular estos errores es asegurar que hay una relación estadísticamente significativa entre el factor y la variable de respuesta.

Los errores estándar se obtienen con el siguiente procedimiento:

1. Calcula las predicciones utilizando los $\beta_0$ y $\beta_1$ encontrados.

In [ ]:
pred = pr(rss.x, X)
pred

2. Idealmente la probabilidad debería ser 100% o 0%. Si alguna predicción no fue absoluta significa que hay incertidumbre. Calcula $p(1-p)$ para todas tus predicciones.

In [ ]:
p = pred*(1-pred)
p

3. Crea una matriz vacía y llena la diagonal con las probabilidades encontradas.

`V = np.diagflat(*p(1-p)*)`

In [ ]:
V = np.diagflat(p)
V

4. Calcula la matriz de covarianza. (Dado que X es la matriz que contiene todos los factores)

`cov = np.linalg.inv(X.T @ V @ X)`

In [ ]:
cov = np.linalg.inv(X.T @ V @ X)
cov

5. Los valores en la diagonal de la matriz de covarianza corresponden a la varianza de los factores. Utiliza los valores de la diagonal para calcular el error estándar.

`se = np.sqrt(np.diag(cov))`

In [ ]:
se = np.sqrt(np.diag(cov))
se

Ahora, revisemos si los estimados de nuestros coeficientes demuestran que hay una relación significativa entre los factores y la respuesta.

Calculamos el estadístico *z*

$$ z_j = \frac{\hat{\beta_j}}{\text{SE}(\hat{\beta_j})} $$

In [ ]:
z = rss.x / se
z

Utilizamos el estadístico *z* para encontrar el *p-value*.

`from scipy.stats import norm`

`p_value = 2 * (1 - norm.cdf(abs(z_statistic)))`

In [ ]:
from scipy.stats import norm

In [ ]:
p_value = 2 * (1 - norm.cdf(abs(z)))
p_value

¿Es significativa la relación de los factores con la variable de respuesta?

Si porque los valores son muy cercanos a 0 o 0

Repite el procedimiento con el factor `student`. 
1. Transforma el factor de {"Yes", "No"} a {1, 0}.
2. Utiliza `minimize` para estimar los coeficientes. Utiliza $\beta_0 = -3.5$ y $\beta_1 = 0.4$ como punto de partida.
3. Calcula el error estándar de tus estimaciones.
   1. Usa tu modelo para encontrar $\hat{p}(X)$
   2. Calcula el error $p(1-p)$
   3. Calcula la matriz de covarianza
   4. Extrae el error estándar
5. Argumenta si los factores son significativos utilizando el *p-value*.
   1. Utiliza el error estándar para calcular el estadístico *z*
   2. Calcula el *p-value*
   3. ¿Son significativos?


In [ ]:
y1 = datos["default"] == "Yes"
x1 = datos["student"]
#chat

# Transformar la columna 'default' en valores numéricos
datos['student_num'] = datos['student'].map({'Yes': 1, 'No': 0}) #transformación a numeros

# Crear la gráfica de dispersión
plt.scatter(datos['balance'], datos['student_num'], alpha=0.5)

# Etiquetas y título
plt.title('Gráfica de dispersión: Balance vs Student transformado')
plt.xlabel('Balance')
plt.ylabel('Student (1 = Yes, 0 = No)')

# Mostrar la gráfica
plt.show()
datos['student_num']

In [ ]:
y1=datos["default"]=="Yes" #y_setosa
x1=datos["student"]
#chat
# Transformar la columna 'default' en valores numéricos
datos['default_num'] = datos['default'].map({'Yes': 1, 'No': 0})

# Crear la gráfica de dispersión
plt.scatter(datos['student'], datos['default_num'], alpha=0.5)

# Etiquetas y título
plt.title('Gráfica de dispersión: Balance vs Default transformado')
plt.xlabel('Balance')
plt.ylabel('Default (1 = Yes, 0 = No)')

# Mostrar la gráfica
plt.show()

In [ ]:
n1 = len(y1)
ones1 = np.ones([n1, 1])
student = np.reshape(datos["student_num"], [n1, 1])
X1 = np.hstack((ones1, student))
y_vector1 = np.reshape(y1, [n1, 1])
def pr1(beta1, X1): 
    return 1 / (1+np.exp(-X1@beta1))
def J1(beta1, X1, y1): 
    epsilon = 1e-12
    n1, p1 = X1.shape
    beta1 = np.reshape(beta1,[p1, 1])
    y_pred1 = pr1(beta1, X1)
    e1 = y1*np.log(y_pred1+epsilon)+(1-y1)*np.log(1-y_pred1+epsilon)
    return -np.sum(e1)
beta1 = [-3.5, .4]
rrs1 = opt.minimize(J1, beta1, args=(X1, y_vector1))
rrs1

In [ ]:
pre1=pr1(rrs1.x, X1)
pre1

In [ ]:
p1=pre1*(1-pre1)
p1

In [ ]:
V1=np.diagflat(p1)
cov1 = np.linalg.inv(X1.T @ V1 @ X1)
cov1

In [187]:
se1 = np.sqrt(np.diag(cov1))
se1

array([222.50814355, 355.07483234])

In [189]:
z1=rrs1.x/se1
z1

array([-0.08840827,  0.00123382])

In [191]:
p_v1=2 * (1 - norm.cdf(abs(z1)))
p_v1

array([0.92955219, 0.99901555])

## Regresión logística múltiple

Considera ahora el caso de múltiples factores. Intentemos predecir si la persona dejará de pagar su crédito utilizando toda la información que tenemos disponible. I.e.

$$ P(\text{default}=1|\text{balance}, \text{income}, \text{student}) = \frac{1}{1 + e^{-(\beta_0 + \beta_1  \text{balance} + \beta_2 \text{income} + \beta_3 \text{student})}} $$

1. Utiliza `minimize` para estimar los coeficientes. Utiliza los siguientes valores como punto de partida:
   - $\beta_0 = -10$
   - $\beta_1 = 0.005$
   - $\beta_2 = 0.003$
   - $\beta_3 = -0.65$
2. Calcula el error estándar de tus estimaciones.
3. Argumenta si los factores son significativos utilizando el *p-value*. 

In [ ]:
balance, student, y
x3=datos["income"]
income=np.reshape((datos["income"])/1000, [n, 1])
ones, balance,income, student
X_T= np.hstack((ones, balance, income, student))
def prediccion3(beta3, X_T): 
    return 1/(1+np.exp(-X_T@beta3))
def J3 (beta3, X_T, y): 
    epsilon=1e-45
    n, p3=X_T.shape
    beta=np.reshape(beta3, [p3, 1])
    y_pred3=prediccion3(beta3, X_T)
    e=y*np.log(y_pred3+epsilon)+(1-y)*np.log(1-y_pred3+epsilon)
    return -np.sum(e)
beta3=[-10, .005, .003, -.65]
rr3=opt.minimize(J3, beta3, args=(X_T, y_vector))
rr3

In [ ]:
pre3=prediccion3(rr3.x, X_T)
p3=pre3*(1-pre3)
V3=np.diagflat(p3)
cov3 = np.linalg.inv(X_T.T @ V3 @ X_T)
se3 = np.sqrt(np.diag(cov3))
z3=rr3.x/se3
p_v3=2 * (1 - norm.cdf(abs(este3)))

¿Cómo sabemos qué tan bueno es el modelo? Hay cuatro posibles casos para un problema de clasificación simple:
- Era sí y se predijo sí. (Verdadero positivo **TP**)
- Era sí y se predijo no. (Falso negativo **FN**)
- Era no y se predijo sí. (Falso positivo **FP**)
- Era no y se predijo no. (Verdadero negativo **TN**)

De esos cuatro casos hay dos donde el modelo es correcto y dos donde el modelo no es correcto.

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*IuymDnZpRlkat0qejE26Nw.png)

1. Menciona dos ejemplos donde consideres que un falso positivo sea un peor resultado que un falso negativo.

2. Menciona dos ejemplos donde consideres que un falso negativo sea un peor resultado que un falso positivo.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3